# OPC UA SERVER HMI DELTA DOP100WS

## Exploracion inicial

In [1]:
from opcua import Client
import pandas as pd

# Conectar al servidor OPC UA
def connect_to_server(url):
    try:
        client = Client(url)
        client.connect()
        print(f"Conectado al servidor OPC UA en {url}")
        return client
    except Exception as e:
        print(f"Error al conectar con el servidor: {e}")
        return None

# Escanear nodos disponibles en el servidor
def scan_nodes(client, node_id="ns=0;i=85"):
    try:
        root_node = client.get_node(node_id)
        print(f"Escaneando nodos bajo {node_id} ({root_node})")
        
        children = root_node.get_children()
        node_data = []

        for child in children:
            browse_name = child.get_browse_name()
            node_id = child.nodeid.to_string()
            display_name = child.get_display_name().Text
            node_data.append({
                "Browse Name": str(browse_name),
                "Node ID": node_id,
                "Display Name": display_name
            })

        df = pd.DataFrame(node_data)
        return df

    except Exception as e:
        print(f"Error al escanear nodos: {e}")
        return None

# Cerrar la conexión con el servidor
def disconnect_server(client):
    try:
        client.disconnect()
        print("Conexión cerrada con el servidor OPC UA")
    except Exception as e:
        print(f"Error al desconectar del servidor: {e}")

# Parámetros de conexión
server_url = "opc.tcp://192.168.8.89:4840"

# Conectar y escanear
client = connect_to_server(server_url)
if client:
    node_df = scan_nodes(client)
    if node_df is not None:
        print("\nNodos escaneados:")
        print(node_df)
        
        # Guardar los nodos en un archivo CSV
        output_file = "opcua_nodes.csv"
        node_df.to_csv(output_file, index=False)
        print(f"Datos de nodos guardados en {output_file}")

    # Desconectar del servidor
    disconnect_server(client)


Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Escaneando nodos bajo ns=0;i=85 (i=85)

Nodos escaneados:
                         Browse Name                   Node ID  \
0            QualifiedName(0:Server)                    i=2253   
1  QualifiedName(1:HMI_DataProvider)  ns=1;s=/HMI_DataProvider   

       Display Name  
0            Server  
1  HMI_DataProvider  
Datos de nodos guardados en opcua_nodes.csv
Conexión cerrada con el servidor OPC UA


Del  resultado de la exploracion inicial podemos determinas las siguientes caracteristicas:
1. ns=1
2. s=/HMI_DataProvider(nodo padre)

## Exploracion de nodos hijos

In [2]:
from opcua import Client

# Conectar al servidor OPC UA
def connect_to_server(url):
    try:
        client = Client(url)
        client.connect()
        print(f"Conectado al servidor OPC UA en {url}")
        return client
    except Exception as e:
        print(f"Error al conectar con el servidor: {e}")
        return None

# Leer el valor de un nodo
def read_node_value(client, node_id):
    try:
        node = client.get_node(node_id)
        value = node.get_value()
        print(f"Valor del nodo {node_id}: {value}")
        return value
    except Exception as e:
        print(f"Error al leer el nodo {node_id}: {e}")
        return None

# Cerrar la conexión
def disconnect_server(client):
    try:
        client.disconnect()
        print("Conexión cerrada con el servidor OPC UA")
    except Exception as e:
        print(f"Error al desconectar del servidor: {e}")


def get_node_info(client, node_id):
    try:
        node = client.get_node(node_id)
        node_class = node.get_node_class()
        print(f"Clase del nodo {node_id}: {node_class}")
        return node_class
    except Exception as e:
        print(f"Error al obtener información del nodo {node_id}: {e}")
        
        
def scan_node_children(client, node_id):
    try:
        node = client.get_node(node_id)
        children = node.get_children()
        print(f"Hijos del nodo {node_id}:")
        for child in children:
            print(f" - {child} ({child.get_display_name().Text})")
        return children
    except Exception as e:
        print(f"Error al escanear los hijos del nodo {node_id}: {e}")
        return None


# Parámetros de conexión
server_url = "opc.tcp://192.168.8.89:4840"
node_id = "ns=1;s=/HMI_DataProvider"  # Reemplazar con el Node ID deseado

# Conectar y leer valor del nodo
client = connect_to_server(server_url)
if client:
   # read_node_value(client, node_id)
    get_node_info(client,node_id)
    scan_node_children(client,node_id)
    disconnect_server(client)


Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Clase del nodo ns=1;s=/HMI_DataProvider: 1
Hijos del nodo ns=1;s=/HMI_DataProvider:
 - ns=1;s=/HMI_DataProvider/Tag1 (Tag1)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad (Adquision_datos_electricidad)
Conexión cerrada con el servidor OPC UA



Determinamos que el HMI_DataProvider es un contenedor porque arroja 1
`node_class = node.get_node_class()`

Si hay hijos de clase Variable (Clase 2):

**Verás sus valores impresos en la salida.**

Si todos los hijos son Object (Clase 1):

**Tendrás que explorar recursivamente hasta encontrar nodos de clase Variable.**

Si no hay hijos:

**Esto significa que Adquision_datos_electricidad es un contenedor vacío o que el servidor no expone más datos.**


In [3]:
# Leer valores de los hijos del nodo
def read_child_nodes(client, parent_node_id):
    try:
        parent_node = client.get_node(parent_node_id)
        children = parent_node.get_children()
        
        for child in children:
            try:
                node_id = child.nodeid.to_string()
                value = child.get_value()
                display_name = child.get_display_name().Text
                print(f"Nodo: {node_id} ({display_name}) -> Valor: {value}")
            except Exception as e:
                print(f"Error al leer el nodo {node_id}: {e}")
    except Exception as e:
        print(f"Error al escanear los hijos del nodo {parent_node_id}: {e}")

# Conectar al servidor y leer hijos
client = connect_to_server(server_url)
if client:
    read_child_nodes(client, "ns=1;s=/HMI_DataProvider")
    disconnect_server(client)


Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Nodo: ns=1;s=/HMI_DataProvider/Tag1 (Tag1) -> Valor: False
Error al leer el nodo ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad: "The attribute is not supported for the specified Node."(BadAttributeIdInvalid)
Conexión cerrada con el servidor OPC UA


Error por que Adquision_datos_electricidad no es un tag, **es un contenedor**

In [62]:
def get_node_class(client, node_id):
    try:
        node = client.get_node(node_id)
        node_class = node.get_node_class()
        print(f"Clase del nodo {node_id}: {node_class}")
    except Exception as e:
        print(f"Error al obtener la clase del nodo {node_id}: {e}")

# Conectar al servidor y verificar la clase del nodo
client = connect_to_server(server_url)
if client:
    get_node_class(client, "ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad")
    disconnect_server(client)


Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Clase del nodo ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad: 1
Conexión cerrada con el servidor OPC UA


Afirmamos que es un contenedor, Procedemos a buscar los tags(nodos) del contenedor **Adquisicion_datos_electricidad**

In [4]:
def explore_children(client, node_id):
    try:
        node = client.get_node(node_id)
        children = node.get_children()
        print(f"Hijos del nodo {node_id}:")
        for child in children:
            print(f" - {child.nodeid.to_string()} ({child.get_display_name().Text})")
        return children
    except Exception as e:
        print(f"Error al explorar los hijos del nodo {node_id}: {e}")
        return None

# Conectar y explorar los hijos del nodo
client = connect_to_server(server_url)
if client:
    explore_children(client, "ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad")
    disconnect_server(client)


Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Hijos del nodo ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad:
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/Consumo_electrico (Consumo_electrico)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VA (VA)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VB (VB)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VC (VC)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VAB (VAB)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VBC (VBC)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VAC (VAC)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/CA (CA)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/CB (CB)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/CC (CC)
Conexión cerrada con el servidor OPC UA


Quiero suponer que el Consumo_electrico es el nodo final veamos

In [5]:
def get_node_class(client, node_id):
    try:
        node = client.get_node(node_id)
        node_class = node.get_node_class()
        print(f"Clase del nodo {node_id}: {node_class}")
    except Exception as e:
        print(f"Error al obtener la clase del nodo {node_id}: {e}")

# Conectar al servidor y verificar la clase del nodo
client = connect_to_server(server_url)
if client:
    get_node_class(client, "ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/Consumo_electrico")
    disconnect_server(client)

Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Clase del nodo ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/Consumo_electrico: 2
Conexión cerrada con el servidor OPC UA


Es un nodo obtenemos su valor

In [6]:
from opcua import Client
import pandas as pd

# Conectar al servidor OPC UA
def connect_to_server(url):
    try:
        client = Client(url)
        client.connect()
        print(f"Conectado al servidor OPC UA en {url}")
        return client
    except Exception as e:
        print(f"Error al conectar con el servidor: {e}")
        return None

# Cerrar la conexión con el servidor
def disconnect_server(client):
    try:
        client.disconnect()
        print("Conexión cerrada con el servidor OPC UA")
    except Exception as e:
        print(f"Error al desconectar del servidor: {e}")

# Obtener la clase del nodo
def get_node_class(client, node_id):
    try:
        node = client.get_node(node_id)
        node_class = node.get_node_class()
        print(f"Clase del nodo {node_id}: {node_class}")
        return node_class
    except Exception as e:
        print(f"Error al obtener la clase del nodo {node_id}: {e}")
        return None

# Explorar hijos de un nodo
def explore_children(client, node_id):
    try:
        node = client.get_node(node_id)
        children = node.get_children()
        print(f"Hijos del nodo {node_id}:")
        for child in children:
            print(f" - {child.nodeid.to_string()} ({child.get_display_name().Text})")
        return children
    except Exception as e:
        print(f"Error al explorar los hijos del nodo {node_id}: {e}")
        return None

# Leer el valor de un nodo
def read_node_value(client, node_id):
    try:
        node = client.get_node(node_id)
        value = node.get_value()
        display_name = node.get_display_name().Text
        print(f"Nodo: {node_id} ({display_name}) -> Valor: {value}")
        return value
    except Exception as e:
        print(f"Error al leer el nodo {node_id}: {e}")
        return None

# Parámetros de conexión
server_url = "opc.tcp://192.168.8.89:4840"
parent_node_id = "ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad"
child_node_id = "ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/Consumo_electrico"

# Conectar al servidor y realizar operaciones
client = connect_to_server(server_url)
if client:
    # Obtener la clase del nodo principal
    get_node_class(client, parent_node_id)
    
    # Explorar hijos del nodo principal
    explore_children(client, parent_node_id)
    
    # Leer el valor del nodo hijo específico
    a=read_node_value(client, child_node_id)
    
    # Desconectar del servidor
    disconnect_server(client)


Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Clase del nodo ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad: 1
Hijos del nodo ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad:
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/Consumo_electrico (Consumo_electrico)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VA (VA)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VB (VB)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VC (VC)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VAB (VAB)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VBC (VBC)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/VAC (VAC)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/CA (CA)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/CB (CB)
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/CC (CC)
Nodo: ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/Consumo_electrico (Consumo_electrico) -> Valor: 1022

Efectivamente obtenemos el dato de electricidad

In [59]:
print(type(a))

<class 'float'>


es de tipo flotante.

Ahora la union de todas las partes, quedaria de la siguiente manera para leer un nodo

In [65]:
from opcua import Client
import pandas as pd

# Conectar al servidor OPC UA
def connect_to_server(url):
    try:
        client = Client(url)
        client.connect()
        print(f"Conectado al servidor OPC UA en {url}")
        return client
    except Exception as e:
        print(f"Error al conectar con el servidor: {e}")
        return None

# Cerrar la conexión con el servidor
def disconnect_server(client):
    try:
        client.disconnect()
        print("Conexión cerrada con el servidor OPC UA")
    except Exception as e:
        print(f"Error al desconectar del servidor: {e}")

# Obtener la clase del nodo
def get_node_class(client, node_id):
    try:
        node = client.get_node(node_id)
        node_class = node.get_node_class()
        print(f"Clase del nodo {node_id}: {node_class}")
        return node_class
    except Exception as e:
        print(f"Error al obtener la clase del nodo {node_id}: {e}")
        return None

# Explorar hijos de un nodo
def explore_children(client, node_id):
    try:
        node = client.get_node(node_id)
        children = node.get_children()
        print(f"Hijos del nodo {node_id}:")
        for child in children:
            print(f" - {child.nodeid.to_string()} ({child.get_display_name().Text})")
        return children
    except Exception as e:
        print(f"Error al explorar los hijos del nodo {node_id}: {e}")
        return None

# Leer el valor de un nodo
def read_node_value(client, node_id):
    try:
        node = client.get_node(node_id)
        value = node.get_value()
        display_name = node.get_display_name().Text
        print(f"Nodo: {node_id} ({display_name}) -> Valor: {value}")
        return value
    except Exception as e:
        print(f"Error al leer el nodo {node_id}: {e}")
        return None

# Parámetros de conexión
server_url = "opc.tcp://192.168.8.89:4840"
parent_node_id = "ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad"
child_node_id = "ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/Consumo_electrico"

# Conectar al servidor y realizar operaciones
client = connect_to_server(server_url)
if client:
    # Obtener la clase del nodo principal
    get_node_class(client, parent_node_id)
    
    # Explorar hijos del nodo principal
    explore_children(client, parent_node_id)
    
    # Leer el valor del nodo hijo específico
    read_node_value(client, child_node_id)
    
    # Desconectar del servidor
    disconnect_server(client)


Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Clase del nodo ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad: 1
Hijos del nodo ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad:
 - ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/Consumo_electrico (Consumo_electrico)
Nodo: ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad/Consumo_electrico (Consumo_electrico) -> Valor: 1014.1410522460938
Conexión cerrada con el servidor OPC UA


Procedemos a hacer script para leer todos los tags de un nodo

In [24]:
from opcua import Client
import pandas as pd

# Conectar al servidor OPC UA
def connect_to_server(url):
    try:
        client = Client(url)
        client.connect()
        print(f"Conectado al servidor OPC UA en {url}")
        return client
    except Exception as e:
        print(f"Error al conectar con el servidor: {e}")
        return None

# Cerrar la conexión con el servidor
def disconnect_server(client):
    try:
        client.disconnect()
        print("Conexión cerrada con el servidor OPC UA")
    except Exception as e:
        print(f"Error al desconectar del servidor: {e}")

# Obtener la clase del nodo
def get_node_class(client, node_id):
    try:
        node = client.get_node(node_id)
        node_class = node.get_node_class()
        print(f"Clase del nodo {node_id}: {node_class}")
        return node_class
    except Exception as e:
        print(f"Error al obtener la clase del nodo {node_id}: {e}")
        return None

# Explorar hijos de un nodo y leer valores
def explore_and_read_children(client, node_id):
    try:
        node = client.get_node(node_id)
        children = node.get_children()
        print(f"Hijos del nodo {node_id}:")
        
        node_data = []
        for child in children:
            try:
                child_node_id = child.nodeid.to_string()
                display_name = child.get_display_name().Text
                value = child.get_value()
                print(f"{display_name}: {value}")
                node_data.append({
                    "Node ID": child_node_id,
                    "Display Name": display_name,
                    "Value": value
                })
            except Exception as e:
                print(f"{child.get_display_name().Text}: Error al leer valor")
                node_data.append({
                    "Node ID": child.nodeid.to_string(),
                    "Display Name": child.get_display_name().Text,
                    "Value": "Error al leer"
                })
        
        # Convertir a DataFrame y retornar
        df = pd.DataFrame(node_data)
        return df

    except Exception as e:
        print(f"Error al explorar los hijos del nodo {node_id}: {e}")
        return None

# Parámetros de conexión
server_url = "opc.tcp://192.168.8.89:4840"
parent_node_id = "ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad"

# Conectar al servidor y realizar operaciones
client = connect_to_server(server_url)
if client:
    # Obtener la clase del nodo principal
    get_node_class(client, parent_node_id)
    
    # Explorar hijos del nodo principal y leer valores
    node_data_df = explore_and_read_children(client, parent_node_id)
    if node_data_df is not None:
        print("\nDatos de los nodos guardados en archivo CSV")
        
        # Guardar los datos en un archivo CSV
        output_file = "opcua_node_data.csv"
        node_data_df.to_csv(output_file, index=False)
        print(f"Datos de los nodos guardados en {output_file}")
    
    # Desconectar del servidor
    disconnect_server(client)


Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Clase del nodo ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad: 1
Hijos del nodo ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad:
Consumo_electrico: 1022.93603515625
VA: 136.79600524902344
VB: 135.2519989013672
VC: 139.57899475097656
VAB: 235.0800018310547
VBC: 239.1060028076172
VAC: 238.8470001220703
CA: 1.3600000143051147
CB: 1.3600000143051147
CC: 0.0

Datos de los nodos guardados en archivo CSV
Datos de los nodos guardados en opcua_node_data.csv
Conexión cerrada con el servidor OPC UA


Anexamos la conexion para hacer la peticion POST a la API en lambda para escribir en dynamoDB


In [25]:
import requests
from opcua import Client
import json
from decimal import Decimal


# Serializador personalizado para convertir Decimal a float
def custom_json_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # Convertir Decimal a float para JSON
    raise TypeError(f"Tipo no serializable: {type(obj)}")


# Conectar al servidor OPC UA
def connect_to_server(url):
    try:
        client = Client(url)
        client.connect()
        print(f"Conectado al servidor OPC UA en {url}")
        return client
    except Exception as e:
        print(f"Error al conectar con el servidor: {e}")
        return None

# Cerrar la conexión con el servidor
def disconnect_server(client):
    try:
        client.disconnect()
        print("Conexión cerrada con el servidor OPC UA")
    except Exception as e:
        print(f"Error al desconectar del servidor: {e}")

# Explorar hijos de un nodo y leer valores
def explore_and_read_children(client, node_id):
    try:
        node = client.get_node(node_id)
        children = node.get_children()
        node_data = []
        for child in children:
            try:
                display_name = child.get_display_name().Text
                value = child.get_value()
                # Convertir los valores flotantes a Decimal
                value = Decimal(str(value)) if isinstance(value, float) else value
                node_data.append({
                    "name": display_name,
                    "value": value
                })
            except Exception as e:
                node_data.append({
                    "name": child.get_display_name().Text,
                    "value": "Error al leer"
                })
        return node_data
    except Exception as e:
        print(f"Error al explorar los hijos del nodo {node_id}: {e}")
        return []

# Validar datos antes de enviar a la API
def validate_node_data(node_data):
    valid_data = []
    for tag in node_data:
        if "name" in tag and "value" in tag:
            valid_data.append(tag)
        else:
            print(f"Datos inválidos encontrados: {tag}")
    return valid_data

# Parámetros del servidor OPC UA
server_url = "opc.tcp://192.168.8.89:4840"
parent_node_id = "ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad"

# URL del endpoint API REST
api_url = "https://zftcgbnpl0.execute-api.us-east-1.amazonaws.com/HMI/electricidad"

# Conectar al servidor OPC UA y enviar datos al endpoint
client = connect_to_server(server_url)
if client:
    try:
        # Leer datos del nodo
        node_data = explore_and_read_children(client, parent_node_id)
        
        # Validar los datos leídos
        valid_node_data = validate_node_data(node_data)
        
        # Verificar si hay datos válidos antes de enviar
        if valid_node_data:
            # Preparar el payload para la API
            payload = {"tags": valid_node_data}
            
            # Imprimir el payload antes de enviarlo
            print("Payload enviado a la API:")
            print(json.dumps(payload, indent=2, default=custom_json_serializer))
            
            # Enviar datos a la API REST
            headers = {"Content-Type": "application/json"}
            response = requests.post(api_url, json=json.loads(json.dumps(payload, default=custom_json_serializer)), headers=headers)
            
            # Mostrar respuesta de la API
            print("Respuesta de la API:")
            print(response.status_code, response.text)
        else:
            print("No se encontraron datos válidos para enviar a la API.")
    finally:
        disconnect_server(client)


Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Payload enviado a la API:
{
  "tags": [
    {
      "name": "Consumo_electrico",
      "value": 1022.93603515625
    },
    {
      "name": "VA",
      "value": 136.78500366210938
    },
    {
      "name": "VB",
      "value": 134.7239990234375
    },
    {
      "name": "VC",
      "value": 138.88299560546875
    },
    {
      "name": "VAB",
      "value": 234.96099853515625
    },
    {
      "name": "VBC",
      "value": 237.64599609375
    },
    {
      "name": "VAC",
      "value": 238.19400024414062
    },
    {
      "name": "CA",
      "value": 1.8200000524520874
    },
    {
      "name": "CB",
      "value": 1.8200000524520874
    },
    {
      "name": "CC",
      "value": 0.0
    }
  ]
}
Respuesta de la API:
200 {"statusCode": 200, "body": "{\"message\": \"Datos almacenados correctamente\", \"tags\": [{\"name\": \"Consumo_electrico\", \"value\": 1022.93603515625}, {\"name\": \"VA\", \"value\": 136.78500366210938

Realizamos la llamada post a la API en Lambda para la escritura en RDS POSTGRESQL, mismo script solo cambia el endpoint y la el tiemstamp.

In [10]:
import requests
from opcua import Client
import json
from decimal import Decimal, ROUND_HALF_UP


# Serializador personalizado para convertir Decimal a float
def custom_json_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # Convertir Decimal a float para JSON
    raise TypeError(f"Tipo no serializable: {type(obj)}")


# Conectar al servidor OPC UA
def connect_to_server(url):
    try:
        client = Client(url)
        client.connect()
        print(f"Conectado al servidor OPC UA en {url}")
        return client
    except Exception as e:
        print(f"Error al conectar con el servidor: {e}")
        return None

# Cerrar la conexión con el servidor
def disconnect_server(client):
    try:
        client.disconnect()
        print("Conexión cerrada con el servidor OPC UA")
    except Exception as e:
        print(f"Error al desconectar del servidor: {e}")

# Explorar hijos de un nodo y leer valores
def explore_and_read_children(client, node_id):
    try:
        node = client.get_node(node_id)
        children = node.get_children()
        node_data = []
        for child in children:
            try:
                display_name = child.get_display_name().Text
                value = child.get_value()

                # Convertir los valores flotantes a Decimal con 2 decimales
                if isinstance(value, float):
                    value = Decimal(str(value)).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)  # Redondeo a 2 decimales
               
                node_data.append({
                    "name": display_name,
                    "value": value
                })
            except Exception as e:
                node_data.append({
                    "name": child.get_display_name().Text,
                    "value": "Error al leer"
                })
        return node_data
    except Exception as e:
        print(f"Error al explorar los hijos del nodo {node_id}: {e}")
        return []

# Validar datos antes de enviar a la API
def validate_node_data(node_data):
    valid_data = []
    for tag in node_data:
        if "name" in tag and "value" in tag:
            valid_data.append(tag)
        else:
            print(f"Datos inválidos encontrados: {tag}")
    return valid_data

# Parámetros del servidor OPC UA
server_url = "opc.tcp://192.168.8.89:4840"
parent_node_id = "ns=1;s=/HMI_DataProvider/Adquision_datos_electricidad"

# URL del endpoint API REST
api_url = "https://zftcgbnpl0.execute-api.us-east-1.amazonaws.com/HMI/electricidad/RDS"

# Conectar al servidor OPC UA y enviar datos al endpoint
client = connect_to_server(server_url)
if client:
    try:
        # Leer datos del nodo
        node_data = explore_and_read_children(client, parent_node_id)
        
        # Validar los datos leídos
        valid_node_data = validate_node_data(node_data)
        
        # Verificar si hay datos válidos antes de enviar
        if valid_node_data:
            # Preparar el payload para la API
            payload = {"tags": valid_node_data}
            
            # Imprimir el payload antes de enviarlo
            print("Payload enviado a la API:")
            print(json.dumps(payload, indent=2, default=custom_json_serializer))
            
            # Enviar datos a la API REST
            headers = {"Content-Type": "application/json"}
            response = requests.post(api_url, json=json.loads(json.dumps(payload, default=custom_json_serializer)), headers=headers)
            
            # Mostrar respuesta de la API
            print("Respuesta de la API:")
            print(response.status_code, response.text)
        else:
            print("No se encontraron datos válidos para enviar a la API.")
    finally:
        disconnect_server(client)


Requested session timeout to be 3600000ms, got 120000ms instead


Conectado al servidor OPC UA en opc.tcp://192.168.8.89:4840
Payload enviado a la API:
{
  "tags": [
    {
      "name": "Consumo_electrico",
      "value": 1026.94
    },
    {
      "name": "VA",
      "value": 135.59
    },
    {
      "name": "VB",
      "value": 134.31
    },
    {
      "name": "VC",
      "value": 138.41
    },
    {
      "name": "VAB",
      "value": 233.21
    },
    {
      "name": "VBC",
      "value": 237.4
    },
    {
      "name": "VAC",
      "value": 236.65
    },
    {
      "name": "CA",
      "value": 1.56
    },
    {
      "name": "CB",
      "value": 1.56
    },
    {
      "name": "CC",
      "value": 0.0
    }
  ]
}
Respuesta de la API:
200 {"statusCode": 200, "body": "{\"message\": \"Datos guardados exitosamente\"}"}
Conexión cerrada con el servidor OPC UA
